<a href="https://colab.research.google.com/github/wlin988/Colab/blob/master/5_1_Pretrained_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [37]:
%cd /content/drive/
!ls -lta

/content/drive
total 8
drwx------ 4 root root 4096 Oct 27 19:14 'My Drive'
drwx------ 2 root root 4096 Oct 27 19:14  .Trash


In [42]:
!cp -r "My Drive"  my_drive

cp: cannot create directory 'my_drive': Operation not supported


In [0]:
!git clone https://github.com/joosthub/PyTorchNLPBook.git

Cloning into 'PyTorchNLPBook'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 7.91 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [15]:
pip install annoy

     |████████████████████████████████| 645kB 4.9MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310411 sha256=9b50d395134c38d942b53c8e09c0733f1e7f96cb2ff566f2278a8d53c26b741a
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy


In [80]:
%cd /content/drive/My Drive/PyTorchNLPBook
!ls

/content/drive/My Drive/PyTorchNLPBook
chapters  data	LICENSE  model_storage	README.md  requirements.txt  SETUP.md


In [82]:
%cd data
!ls -lta

[Errno 2] No such file or directory: 'data'
/content/drive/My Drive/PyTorchNLPBook/data
total 6
-rw------- 1 root root 1572 Oct 27 19:25 download.py
-rw------- 1 root root 1951 Oct 27 19:25 get-all-data.sh
-rw------- 1 root root 1929 Oct 27 19:25 README.md


In [48]:

%cd /content/drive/"My Drive"/PyTorchNLPBook/chapters/chapter_5/data
!pwd
!ls -lta

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data
/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data
total 9
-rwx------ 1 root root 1961 Oct 27 19:51 get-all-data.sh
drwx------ 3 root root 4096 Oct 27 19:43 Drive
-rw------- 1 root root 1572 Oct 27 19:25 download.py
-rw------- 1 root root  572 Oct 27 19:25 README.md


In [87]:
!chmod 777 get-all-data.sh
!ls -lta

total 30
drwx------ 2 root root 4096 Oct 27 20:15 nmt
drwx------ 2 root root 4096 Oct 27 20:15 ag_news
drwx------ 2 root root 4096 Oct 27 20:15 books
drwx------ 2 root root 4096 Oct 27 20:15 surnames
drwx------ 2 root root 4096 Oct 27 20:15 yelp
-rwx------ 1 root root 1961 Oct 27 20:15 get-all-data.sh
drwx------ 3 root root 4096 Oct 27 20:14 Drive
-rw------- 1 root root 1572 Oct 27 19:25 download.py
-rw------- 1 root root 1929 Oct 27 19:25 README.md


In [86]:
! ./get-all-data.sh

Trying to fetch ./yelp/raw_train.csv
12536it [00:04, 3051.49it/s]
Trying to fetch ./yelp/raw_test.csv
848it [00:00, 3929.77it/s]
Trying to fetch ./yelp/reviews_with_splits_lite.csv
1217it [00:00, 2284.57it/s]
Trying to fetch ./surnames/surnames.csv
6it [00:00, 1388.54it/s]
Trying to fetch ./surnames/surnames_with_splits.csv
8it [00:00, 2389.75it/s]
Trying to fetch ./books/frankenstein.txt
14it [00:00, 2544.21it/s]
Trying to fetch ./books/frankenstein_with_splits.csv
109it [00:00, 3043.70it/s]
Trying to fetch ./ag_news/news.csv
188it [00:00, 3395.77it/s]
Trying to fetch ./ag_news/news_with_splits.csv
208it [00:00, 3943.28it/s]
Trying to fetch ./nmt/eng-fra.txt
292it [00:00, 1586.96it/s]
Trying to fetch ./nmt/simplest_eng_fra.csv
30it [00:00, 1801.81it/s]


In [51]:
!mkdir glove
!pwd
!ls

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data
ag_news  download.py  get-all-data.sh  nmt	  surnames
books	 Drive	      glove	       README.md  yelp


In [54]:
%cd glove
!ls

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data/glove


In [55]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-10-27 19:57:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-10-27 19:57:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-10-27 19:57:33--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [58]:
!ls
!unzip glove.6B.zip

glove.6B.zip
Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [62]:
!pwd
%cd ../..
!ls

/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5/data/glove
/content/drive/My Drive/PyTorchNLPBook/chapters/chapter_5
5_1_Pretrained_Embeddings.ipynb  5_3_doc_classification  README.md
5_2_CBOW			 data


In [63]:
%run 5_1_Pretrained_Embeddings.ipynb

Building Index!
Finished!
man : he :: woman : she
man : he :: woman : never
fly : plane :: sail : ship
fly : plane :: sail : vessel
cat : kitten :: dog : puppy
cat : kitten :: dog : toddler
cat : kitten :: dog : sleds
blue : color :: dog : cat
blue : color :: dog : animal
blue : color :: dog : breed
leg : legs :: hand : fingers
leg : legs :: hand : ears
leg : legs :: hand : stick
toe : foot :: finger : hand
toe : foot :: finger : attached
toe : foot :: finger : apart
talk : communicate :: read : instructions
talk : communicate :: read : communicating
talk : communicate :: read : transmit
blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator
man : king :: woman : queen
man : king :: woman : monarch
man : king :: woman : throne
man : doctor :: woman : nurse
man : doctor :: woman : physician
fast : fastest :: small : smallest
fast : fastest :: small : largest
fast : fastest :: small : among


In [0]:
#import os
#from getpass import getpass
#import urllib

#cmd = 'git clone https://github.com/joosthub/PyTorchNLPBook.git'
#os.system(cmd)

In [0]:
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

In [0]:
class PreTrainedEmbeddings(object):
    """ A wrapper around pre-trained word vectors and their use """
    def __init__(self, word_to_index, word_vectors):
        """
        Args:
            word_to_index (dict): mapping from word to integers
            word_vectors (list of numpy arrays)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("Building Index!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("Finished!")
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """Instantiate from pre-trained vector file.
        
        Vector file should be of the format:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        Args:
            embedding_file (str): location of the file
        Returns: 
            instance of PretrainedEmbeddigns
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
                
        return cls(word_to_index, word_vectors)
    
    def get_embedding(self, word):
        """
        Args:
            word (str)
        Returns
            an embedding (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]

    def get_closest_to_vector(self, vector, n=1):
        """Given a vector, return its n nearest neighbors
        
        Args:
            vector (np.ndarray): should match the size of the vectors 
                in the Annoy index
            n (int): the number of neighbors to return
        Returns:
            [str, str, ...]: words that are nearest to the given vector. 
                The words are not ordered by distance 
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_and_print_analogy(self, word1, word2, word3):
        """Prints the solutions to analogies using word embeddings

        Analogies are word1 is to word2 as word3 is to __
        This method will print: word1 : word2 :: word3 : word4
        
        Args:
            word1 (str)
            word2 (str)
            word3 (str)
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)

        # now compute the fourth word's embedding!
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship

        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = set([word1, word2, word3])
        closest_words = [word for word in closest_words 
                             if word not in existing_words] 

        if len(closest_words) == 0:
            print("Could not find nearest neighbors for the computed vector!")
            return
        
        for word4 in closest_words:
            print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

In [68]:
embeddings = PreTrainedEmbeddings.from_embeddings_file('data/glove/glove.6B.100d.txt')

Finished!
Building Index!
Finished!


In [69]:
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : never


In [70]:
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship
fly : plane :: sail : vessel


In [71]:
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

cat : kitten :: dog : puppy
cat : kitten :: dog : toddler
cat : kitten :: dog : sleds


In [72]:
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

blue : color :: dog : cat
blue : color :: dog : animal
blue : color :: dog : breed


In [73]:
embeddings.compute_and_print_analogy('leg', 'legs', 'hand')

leg : legs :: hand : fingers
leg : legs :: hand : ears
leg : legs :: hand : stick


In [74]:
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

toe : foot :: finger : hand
toe : foot :: finger : attached
toe : foot :: finger : apart


In [75]:
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

talk : communicate :: read : instructions
talk : communicate :: read : communicating
talk : communicate :: read : transmit


In [76]:
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator


In [77]:
embeddings.compute_and_print_analogy('man', 'king', 'woman')

man : king :: woman : queen
man : king :: woman : monarch
man : king :: woman : throne


In [78]:
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician


In [79]:
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')

fast : fastest :: small : smallest
fast : fastest :: small : largest
fast : fastest :: small : among
